In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [14]:
import pandas as pd

base_dir = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/"

# abundance table
data = pd.read_csv(f"{base_dir}data/growings_microbiome.csv", sep=",", header=0)

len(data)

360

In [16]:
target = "ch4_g_day2_1v3"

# first column = sample_id, next 158 columns = microbiome (log-transformed)
micro_cols = data.columns[1 : 1+158]

model_df = data.dropna(subset=[target]).copy()  # drop rows where y is NaN

X = model_df[micro_cols]
y = model_df[target]

# ------------------------------------------------------------------
# 3.  Train/test split
# ------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# ------------------------------------------------------------------
# 4.  Standardize (fit on training, apply to test)
# ------------------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print(f"X-train shape: {X_train_scaled.shape}, X-test shape: {X_test_scaled.shape}")

X-train shape: (254, 158), X-test shape: (64, 158)


In [18]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score

lasso = LassoCV(cv=5, random_state=42).fit(X_train_scaled, y_train)
y_pred = lasso.predict(X_test_scaled)

print("R² score:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))


/data/Genetics/analysis/R1681_OviSeq/Dermot/envs/dermo_python_env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.048148399089314, tolerance: 0.49586348083026344
  model = cd_fast.enet_coordinate_descent_gram(
/data/Genetics/analysis/R1681_OviSeq/Dermot/envs/dermo_python_env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6320204856460805, tolerance: 0.49586348083026344
  model = cd_fast.enet_coordinate_descent_gram(
/data/Genetics/analysis/R1681_OviSeq/Dermot/envs/dermo_python_env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8331715034109948, tolerance

R² score: 0.2793915064436069
MSE: 12.395989246696349


/data/Genetics/analysis/R1681_OviSeq/Dermot/envs/dermo_python_env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8569999273440772, tolerance: 0.4657617405023897
  model = cd_fast.enet_coordinate_descent_gram(


In [19]:
selected = np.array(micro_cols)[lasso.coef_ != 0]
print(len(selected), "genera retained")


51 genera retained
